In [1]:
import numpy as np
import pandas as pd
import sys
import os
import shutil

<h2>데이터 전처리<h2>

In [2]:
def read_text_files_to_dataframe(folder_path):
    
    latitude = []
    longitude = []
    date = []
    time = []

    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            
            # 텍스트 파일을 DataFrame에 추가

            with open(file_path, 'r') as file:
                data = file.read()

            data = data.split('\n')[6:-1]
            for i in range(len(data)):
                line = data[i].split(',')
                latitude.append(line[0])
                longitude.append(line[1])
                date.append(line[5])
                time.append(line[6])

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    return df

In [3]:
folder_path = "C:\\Users\\sk002\\OneDrive\\바탕 화면\\학교\\Yoodori\\Geolife Trajectories 1.3\\Data\\000\\Trajectory"

dataframe = read_text_files_to_dataframe(folder_path)

dataframe

,latitude,longitude,datetime
0,39.984702,116.318417,2008-10-23 02:53:00
1,39.984586,116.316716,2008-10-23 02:54:00
2,39.984485,116.314426,2008-10-23 02:55:00
3,39.984696,116.312921,2008-10-23 02:56:00
4,39.984656,116.310660,2008-10-23 02:57:00
...,...,...,...
15118,39.996230,116.326576,2009-07-05 07:41:00
15119,39.999069,116.328971,2009-07-05 07:42:00
15120,39.999975,116.327367,2009-07-05 07:43:00
15121,39.999974,116.327326,2009-07-05 07:44:00


<h2>의미장소 추출<h2>

In [4]:
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
import folium

In [17]:
def gmeans_(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selected_columns = ['latitude', 'longitude']
    result_list = df[selected_columns].values.tolist()
    
    gmeans_instance = gmeans(result_list).process()

    # Extract clustering results: clusters and their centers
    clusters = gmeans_instance.get_clusters()
    centers = gmeans_instance.get_centers()

    return clusters, centers
    

In [6]:
def map(centers, i):

    # 예제 데이터 생성 (서울의 위도, 경도)
    locations = centers

    # 지도의 중심 좌표 설정
    map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

    # Folium 맵 생성
    mymap = folium.Map(location=map_center, zoom_start=12)

    # 위치 데이터를 지도에 마커로 표시
    for loc in locations:
        folium.Marker(location=loc, popup='Location').add_to(mymap)

    # 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
    file_name = f'map{i}.html'
    mymap.save(file_name)


In [7]:
def fileReader(folder_path, filename):
    latitude = []
    longitude = []
    date = []
    time = []
    file_path = os.path.join(folder_path, filename)
            
    # 텍스트 파일을 DataFrame에 추가

    with open(file_path, 'r') as file: 
        data = file.read()

    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])
        longitude.append(line[1])
        date.append(line[5])
        time.append(line[6])

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    return df

In [8]:
def app(folder_path):
    
    j = 0
    data_dict = {}
    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):

        df = fileReader(folder_path, filename)

        df['latitude'] = df['latitude'].astype(float)
        df['longitude'] = df['longitude'].astype(float)
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
        df['datetime'] = df['datetime'].dt.floor('T')
        df = df.drop(['date', 'time'], axis=1)
        df = df.drop_duplicates(['datetime'], ignore_index=True)

        clusters, centers = gmeans(df)

        data_dict[j] = pd.DataFrame({"clusters":clusters, "centers":centers})
        
        for k in range(len(data_dict[j].clusters)):
            if (len(data_dict[j].clusters[k]) < 6):
                data_dict[j].drop(index=k, inplace=True)
        data_dict[j] = data_dict[j].sort_index(axis=1)

        map(centers, j)
        
        j += 1
    return data_dict

In [9]:
#folder_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory'
#data_dict = app(folder_name)

In [18]:
def fileReader(filename):

    latitude = []   # 위도
    longitude = []  # 경도
    date = []       # 날짜
    time = []       # 시간

    with open(filename, 'r') as file:
        data = file.read()

    # 데이터에 불필요한 부분 제거
    # 추후 데이터 형식에 따라 수정 필요 *
    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])    # 위도
        longitude.append(line[1])   # 경도
        date.append(line[5])        # 날짜
        time.append(line[6])        # 시간

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    return df

In [19]:
def gmeansFit(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selectedColumns = ['latitude', 'longitude']
    resultList = df[selectedColumns].values.tolist()    # 리스트로 변환
    
    gmeansInstance = gmeans(resultList).process()       # 클러스터링

    centers = gmeansInstance.get_centers()              # 클러스터의 중심 (의미장소)
    clusters = gmeansInstance.get_clusters()            # 분류된 클러스터


    return clusters, centers

In [20]:
def gmeansFunc(file_name):
    
    j = 0
    data_dict = {}

    df = fileReader(file_name)

    # 위도 경도 데이터 형식 변경
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    # 날짜, 시간 데이터 병합
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    # 1분 단위로 데이터 병합
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    # 의미장소 추출
    clusters, centers = gmeansFit(df)
        
    # 딕셔너리 형식으로 데이터 저장
    data_dict[j] = pd.DataFrame({"clusters":clusters, "centers":centers})
        
    # 클러스터가 10개 미만인 의미장소 제거
    for k in range(len(data_dict[j].clusters)):
        if (len(data_dict[j].clusters[k]) < 10):
            data_dict[j].drop(index=k, inplace=True)
    data_dict[j] = data_dict[j].sort_index(axis=1)
        
    j += 1
    return data_dict

In [21]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory\20090403011657.txt'
data_dict = gmeansFunc(file_name)

TypeError: list indices must be integers or slices, not list

<h2>서버에 올릴 코드<h2>

In [6]:
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
import numpy as np
import pandas as pd
import os
import folium

In [2]:
def fileReader(filename):

    latitude = []   # 위도
    longitude = []  # 경도
    date = []       # 날짜
    time = []       # 시간

    with open(filename, 'r') as file:
        data = file.read()

    # 데이터에 불필요한 부분 제거
    # 추후 데이터 형식에 따라 수정 필요 *
    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])    # 위도
        longitude.append(line[1])   # 경도
        date.append(line[5])        # 날짜
        time.append(line[6])        # 시간

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    return df

In [3]:
# 의미장소 추출
def gmeansFit(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selectedColumns = ['latitude', 'longitude']
    resultList = df[selectedColumns].values.tolist()    # 리스트로 변환
    
    gmeansInstance = gmeans(resultList).process()       # 클러스터링

    centers = gmeansInstance.get_centers()              # 클러스터의 중심 (의미장소)
    clusters = gmeansInstance.get_clusters()            # 분류된 클러스터


    return clusters, centers

In [4]:
def gmeansFunc(filename):
    
    j = 0
    df = fileReader(filename)

    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    clusters, centers = gmeansFit(df)

    data_df = pd.DataFrame({"clusters":clusters, "centers":centers})
        
    for k in range(len(data_df.clusters)):
        if (len(data_df.clusters[k]) < 10):
            data_df.drop(index=k, inplace=True)
    data_df = data_df.sort_index(axis=1)
    data_df = data_df.reset_index(drop=True)

    # map(centers, j)
        
    j += 1
    return data_df

In [7]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory\20090403011657.txt'
data_df = gmeansFunc(file_name)

In [8]:
data_df

,centers,clusters
0,"[39.89846146153847, 116.38943146153848]","[1075, 1076, 1077, 1078, 1079, 1080, 1081, 108..."
1,"[39.9097909090909, 116.40469272727273]","[1037, 1038, 1039, 1040, 1041, 1042, 1043, 104..."
2,"[39.978874235294114, 116.46222988235294]","[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
3,"[39.98576385, 116.41272174999999]","[89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 1..."
4,"[40.043579, 116.60664570000002]","[119, 120, 121, 122, 123, 124, 125, 126, 127, ..."
5,"[40.00945831578947, 116.31558689473685]","[488, 493, 495, 496, 497, 498, 500, 501, 502, ..."
6,"[40.00961204081633, 116.31509499999997]","[43, 321, 366, 377, 380, 385, 391, 396, 399, 4..."
7,"[40.00953293333334, 116.31517493333335]","[227, 230, 340, 390, 411, 412, 419, 421, 433, ..."
8,"[40.00950649999999, 116.3150521875]","[219, 392, 393, 402, 507, 518, 541, 558, 604, ..."
9,"[40.0096832368421, 116.31503742105266]","[238, 322, 368, 378, 379, 395, 409, 414, 417, ..."


In [12]:
data_df.clusters.values.tolist()

[[1075,
  1076,
  1077,
  1078,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1088,
  1089],
 [1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047],
 [101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  152,
  153,
  154,
  155,
  156,
  157,
  158],
 [89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166],
 [119, 120, 121, 122, 123, 124, 125, 126, 127, 128],
 [488,
  493,
  495,
  496,
  497,
  498,
  500,
  501,
  502,
  503,
  515,
  516,
  522,
  523,
  524,
  594,
  602,
  648,
  649],
 [43,
  321,
  366,
  377,
  380,
  385,
  391,
  396,
  399,
  400,
  403,
  404,
  431,
  438,
  451,
  465,
  467,
  468,
  475,
  480,
  484,
  485,
  542,
  543,
  544,
  546,
  547,
  551,
  554,
  557,
  572,
  573,
  576,
  581,
  582,
  589,
  619,
  654,
  655,
  686,
  687,
  690,
  714,
  753,
  798,
  816,
  1155,
  1166,
  1169],
 [227, 230, 340, 390, 411, 412, 419, 421, 4

In [14]:
import folium

# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap
